In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import numpy as np
import h5py
import networkx as nx

from epistasis.models.linear import EpistasisLinearRegression
from epistasis.simulate import LinearSimulation, NonlinearSimulation
from epistasis.stats import pearson

from latticeproteins.sequences import RandomSequence
from latticeproteins.conformations import Conformations

from latticegpm import LatticeGenotypePhenotypeMap
from latticegpm.thermo import LatticeThermodynamics
from latticegpm.search import adaptive_walk, get_lowest_confs

from gpmap.evolve.models import fixation
from gpmap.graph.paths import path_probabilities, paths_and_probabilities

/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
def calculate_phi(model):
    """Calculate the fraction of the phenotypes explained by (statistical) epistasis."""
    # Quick check.
    if model.model_type != "global":
        raise Exception("Argument must be a global model.")
    model.gpm.phenotype_type = "stabilities"
    r2 = [0]
    for i in range(1,9):
        em = model.epistasis.get_orders(*list(range(i+1)))
        sim = LinearSimulation.from_coefs(
            model.gpm.wildtype,
            model.gpm.mutations,
            em.labels,
            em.values,
            model_type=model.model_type)
        r = pearson(sim.phenotypes, model.gpm.phenotypes)**2
        r2.append(r)
    fx = [r2[i] - r2[i-1] for i in range(2, len(r2))]
    fx = [1 - sum(fx)] + fx
    return fx

In [3]:
with open("sequences-2.txt", "r") as f:
    sequences = []
    for line in f.readlines():
        s = line.strip().split(",")
        sequences.append(s)

In [4]:
for i in range(len(sequences)):
    seqs = sequences[i]
    wildtype = seqs[0]
    mutant = seqs[1]
    c = Conformations(12, database_dir="database")

    # Get lowest conformation
    cs = get_lowest_confs(wildtype, 3, database="database")

    # Save sequences
    #sequences[current] = np.array([wildtype, mutant] , dtype="|S" + str(len(wildtype)))

    # Create a genotype-phenotype map for the given sequences (given native target)
    gpm = LatticeGenotypePhenotypeMap.from_mutant(wildtype, mutant, c, target_conf=cs[0])
    #gpm.set_partition_confs([cs[0], "UUUUUUUUUUU"])
    gpm.phenotype_type = "nativeEs"

    # Calculate the phi
    gmodel = EpistasisLinearRegression.from_gpm(gpm, order=8, model_type="global")
    gmodel.fit()

    phi = calculate_phi(gmodel)
    phis = [str(p) for p in phi]
    with open("phi1.csv", "a") as f:
        out = ",".join(seqs + phis)
        f.write(out + "\n")

/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is depre

In [ ]:
for i in range(len(sequences)):
    seqs = sequences[i]
    wildtype = seqs[0]
    mutant = seqs[1]
    c = Conformations(12, database_dir="database")

    # Get lowest conformation
    cs = get_lowest_confs(wildtype, 3, database="database")

    # Save sequences
    #sequences[current] = np.array([wildtype, mutant] , dtype="|S" + str(len(wildtype)))

    # Create a genotype-phenotype map for the given sequences (given native target)
    gpm = LatticeGenotypePhenotypeMap.from_mutant(wildtype, mutant, c, target_conf=cs[0])
    gpm.set_partition_confs(cs[0:2])
    gpm.phenotype_type = "stabilities"

    # Calculate the phi
    gmodel = EpistasisLinearRegression.from_gpm(gpm, order=8, model_type="global")
    gmodel.fit()

    phi = calculate_phi(gmodel)
    phis = [str(p) for p in phi]
    with open("full-state-2state-phi.csv", "a") as f:
        out = ",".join(seqs + phis)
        f.write(out + "\n")

In [11]:
for i in range(len(sequences)):
    seqs = sequences[i]
    wildtype = seqs[0]
    mutant = seqs[1]
    c = Conformations(12, database_dir="database")

    # Get lowest conformation
    cs = get_lowest_confs(wildtype, 3, database="database")

    # Save sequences
    #sequences[current] = np.array([wildtype, mutant] , dtype="|S" + str(len(wildtype)))

    # Create a genotype-phenotype map for the given sequences (given native target)
    gpm = LatticeGenotypePhenotypeMap.from_mutant(wildtype, mutant, c, target_conf=cs[0])
    gpm.set_partition_confs(cs)
    gpm.phenotype_type = "stabilities"

    # Calculate the phi
    gmodel = EpistasisLinearRegression.from_gpm(gpm, order=8, model_type="global")
    gmodel.fit()

    phi = calculate_phi(gmodel)
    phis = [str(p) for p in phi]
    with open("full-state-3state-phi.csv", "a") as f:
        out = ",".join(seqs + phis)
        f.write(out + "\n")

/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/home/zsailer/virtual/py3/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is depre